# Feature Space Partition
Feature space partition based on class labels. In the process, it produces difficult-to-classify and easy-to-classify sub-reagions.

**Table of Contents**

- [Feature Space Partiotion](#Feature-Space-Partition)
  - [Syntax](#Syntax)
  - [Input Arguments](#Input-Arguments)
  - [Output Arguments](#Output-Arguments)
  - [Initial settings](#Initial-settings)
- [While X is not empty do](#While-X-is-not-empty-do)
  - [Segment dataset X into K clusters using k-means](##Segment-dataset-X-into-K-clusters-using-k-means)
    - [Evaluate each cluster](###Evaluate-each-cluster)
    - [Calculate the classification error](###Calculate-the-classification-error)
  - [Is there any homogenous region?](#Is-there-any-homogenous-region)
    - [YES, there is homogenous region](#YES,-there-is-homogenous-region)
      - [Set them aside from X and y and set k=initial_k](#Set-them-aside-from-X-and-y-and-set-k=initial_k)
    - [NO, there is no homogenous region](#NO,-there-is-no-homogenous-region)
      - [If it isn't the first iteration, update the separability threshold](#If-it-isn't-the-first-iteration,-update-the-separability-threshold)
      - [Is there any separable region?](#Is-there-any-separable-region-?)
        - [YES, there is separable region. Set k = k+1](#YES,-there-is-separable-region.-Set-k-=-k+1)
        - [NO, there is no separable region. Set X to empty](#NO,-there-is-no-separable-region.-Set-X-to-empty)
  - [Final settings](#Final-settings)

## Syntax
```python
    Output = Feature_Space_Partition(X, y, opt)
    Output = Feature_Space_Partition(X, y)
```

## Input Arguments
  - **X**: N x d numeric matrix. Each row of **X** represents an observation, while each column corresponds to a feature.
  - **y**: N x 1 numeric vector. The i-th row of **y** is the class label of the i-th observation given in **X**. Furthermore, each $y_{i} \in [1,2,..., n_{CL}]$, where $n_{CL}$ is the number of unique class labels
  - opt: Struct array containing optional settings (default values are provided).

## Output Arguments
  - Output: Struct array containing the results of the partitioning process.

## Initial settings


In [ ]:
tic = time.time()
#define default values for optional parameters
defaultOpt = {
    'initial_k': 1,                     # The starting value for k-means
    'p_parameter': 0.05,                # The p-almost homogeneous parameter
    'h_threshold': 0,                   # This variable's role is to filter homogeneous clusters, considering only those with more than h_threshold observations
    'dm_case': 2,                       # The case for the divergence measure function
    'dm_threshold': 0.5,                # The threshold for the divergence measure (separability parameter)
    'update_dm_threshold': True,        # Flag to update divergence measure threshold
    'return_full_dm': False,            # Flag to determine whether to return the complete divergence measure matrix
    'return_full_history': False,       # Flag to determine whether to return the complete history of the segmentation process
    'iteration_threshold': 2e6,         # The maximum number of iterations allowed
    'rng_default': False                # For reproducibility, use True
}
#update default values with user specified values
if opt is not None:
    defaultOpt.update(opt)
opt = defaultOpt

#Define initial values
k = opt['initial_k']    #Current value of k for k-means
i = 0                   #iteration counter
rmi = 0                 #The iteration counterfor when cluster removal occured
nCl = len(np.unique(y)) #Number of unique class labels
N = X.shape[0]          #Number of observations
p_parameter = opt['p_parameter'] #The p-almost homogeneous parameter
dm_threshold = opt['dm_threshold'] #The divergence measure threshold
sum2 = 0                #This variable is employed in the calculation of the classification 
# Initialize the struct array rmH and H
# In rmH, we keep a history of the segmentation process of iterations where cluster removal occurred.
# In H, we have the complete history of the segmentation process.
rmH, H = initializeStructures(opt)
rmH_list = []
H_list = []

# While X is not empty do

In [ ]:
while X.shape[0] > 0:
    i += 1
    if opt['return_full_history']:
        #Start measuring the loop execution time
        tStart_while = time.time()
        H['k'] = k #Save the current value of k
        H['i'] = i #Save the current iteration number
    Indices_Num = np.zeros(k)
    Indices = {}
    for j in range(k):
        Indices[j] = []
    ClassLabel_Num = np.zeros((k,nCl))
    ClassLabel_Indices = {}
    ClassLabel_Proportion = np.zeros((k,nCl))
    for j in range(k):
        ClassLabel_Indices[j] = []
    DominantClassLabel = np.zeros(k)
    DominantClassLabel_Proportion = np.zeros(k)
    nonDominantClassLabelNumber = np.zeros(k)
    dm = np.array([])
    rmXidx = np.array([])
    sum1 = np.array([])


## Segment dataset X into K clusters using k-means

In [ ]:
       if opt['rng_default']:
            random.seed(0)
        kmeans = KMeans(n_clusters=k,max_iter= 1000, n_init=1).fit(X)
        idx = kmeans.labels_
        C = kmeans.cluster_centers_

### Evaluate each cluster

In [ ]:
        for c in range(0,k):
                    #Indices of the observations in X into cluster c
                    Indices[c] = np.where(idx == c)[0]
                    #Number of observations in X into cluster c
                    Indices_Num[c] = len(Indices[c])
                    #Class labels of the observations in X into cluster c
                    yc = y[Indices[c]]
                    #For each class label, do>
                    for l in range(0,nCl):
                        aux = l + 1
                        ClassLabel_Indices[c,l] = np.where(yc == aux)[0]
                        ClassLabel_Num[c,l] = len(ClassLabel_Indices[c,l])
                        ClassLabel_Proportion[c,l] = ClassLabel_Num[c,l] / Indices_Num[c]


### Calculate the classification error

Cluster classification error
- Let $X_{c}$ and yc be the observations and class labels in cluster c
- Let pc be the proportion of the dominant class label in cluste c
- Let lc be the dominant class label in cluster c
- N the oritional number of observations
- $N_{c}$ the number of observations in cluster c

In [ ]:
            max_value = np.max(ClassLabel_Proportion[c])
            max_index = np.argmax(ClassLabel_Proportion[c])
            pc = max_value
            lc = max_index
            # print(f'max_value: {max_value}')
            # print(f'max_index: {max_index}')
            DominantClassLabel_Proportion[c] = pc
            DominantClassLabel[c] = lc
            nonDominantClassLabelNumber[c] = 0
            for x in range (len(yc)):
                if yc[x] != lc:
                    nonDominantClassLabelNumber[c] += 1
        ClusterClassificationError = np.divide(nonDominantClassLabelNumber, Indices_Num)

Classification error

$ClassificationError = sum2 + \sum_{c=1}^{k} \frac{ClusterClassificationError(c) * N_{c}}{N} = sum2 \sum_{c=1}^{k}\frac{sum(yc \ne lc)}{N}$


In [ ]:
        ClassificationError = sum2 + sum(nonDominantClassLabelNumber) / N


## Is there any homogenous region?

**Definition D1** [p-almost homogenous]. $\R_{c}$ is said to be p-almost-homogeneous if at least $(100-p)\%$ of the observations  are labeled as belonging to the same class.
Note: We are only considering homogeneous clusters in which the number of observations is greater than h_threshold.


In [ ]:
        vec = []
        for j in range(k):
            aux = []
            for b in range(nCl):
                if (ClassLabel_Proportion[j,b] >= 1 - p_parameter) and Indices_Num[j] >= opt['h_threshold']:
                    aux.append(True)
                    break
                else:
                    aux.append(False)
            vec.append(np.max(aux))

### YES, there is homogenous region

In [ ]:
        if sum(vec) > 0:


#### Set them aside from X and y and set k=initial_k


In [ ]:
            rmCidx = np.where(vec)[0]
            for aux in rmCidx:
                rmXidx = np.concatenate((rmXidx, Indices[aux]))
            rmC = C[rmCidx, :]
            rmXidx = rmXidx.astype(int)
            
            rmNonDominantClassLabelNumberbyN = nonDominantClassLabelNumber[rmCidx] / N
            sum1 = sum(rmNonDominantClassLabelNumberbyN)
            sum2 = sum2 + sum1

            rmi = rmi + 1
            rmH['i'].append(i)
            rmH['k'].append(k)
            rmH['C'].append(C)
            rmH['rmCidx'].append(rmCidx)
            rmH['rmC'].append(rmC)
            rmH['dm_threshold'].append(dm_threshold)
            rmH['dm'].append(dm)
            rmH['rmDominantClassLabel'].append(DominantClassLabel[vec])
            rmH['rmDominantClassLabel_Proportion'].append(DominantClassLabel_Proportion[vec])
            rmH['sum1'].append(sum1)
            rmH['rmXidx'].append(rmXidx)
            rmH_list.append(rmH.copy())

            rmH, _ = initializeStructures(opt)

            X = np.delete(X, rmXidx, axis=0)
            y = np.delete(y, rmXidx, axis=0)
            k = opt['initial_k']

### NO, there is no homogenous region

In [ ]:
        else:

#### If it isn't the first iteration, update the separability threshold


dm_threshold = $\frac{H(i-1).dm_threshold}{\sqrt{\frac{H(i).ClassificationError}{H(i-1).ClassificationError}}}$

In [ ]:
            if i > 1 and opt['update_dm_threshold'] and ClassificationError != 0 and ClassificationError_previous != 0:
                #print("Update dm_threshold")
                dm_threshold = dm_threshold_previous / np.sqrt(ClassificationError / ClassificationError_previous)
                #print(dm_threshold)
            

### Is there any separable region?

**Definition D2** [s-separable].  $\R_{c}$ is said to be s-separable if the divergence measure between two probability distribution function (pdf) is equal or greater than a chosen threshold.


In [ ]:
            cst, dm = check_separability_criteria(k,nCl, dm_threshold, X, ClassLabel_Num, ClassLabel_Indices, Indices, opt)


#### YES, there is separable region. Set k = k+1

In [ ]:
            if cst > 0:
                #print("There is separable region")
                k += 1
                if opt['update_dm_threshold']:
                    dm_threshold_previous = dm_threshold

#### NO, there is no separable region. Set X to empty

In [ ]:
            else: 
                rmNonDominantClassLabelNumberbyN = nonDominantClassLabelNumber / N
                sum1 = sum(rmNonDominantClassLabelNumberbyN)
                sum2 = sum2 + sum1
                rmCidx = np.arange(k)
                rmC = C
                rmXidx = np.arange(X.shape[0])
                rmH['i'] = i
                rmH['k'] = k
                rmH['C'] = C
                rmH['rmCidx'] = rmCidx
                rmH['rmC'] = rmC
                rmH['dm_threshold'] = dm_threshold
                rmH['dm'] = dm
                rmH['rmDominantClassLabel'] = DominantClassLabel
                rmH['rmDominantClassLabel_Proportion'] = DominantClassLabel_Proportion
                rmH['rmXidx'] = rmXidx
                rmH['sum1'] = sum1

                rmH_list.append(rmH)

                
                rmi += 1
                X = np.array([])
                y = np.array([])

### Final settings

In [ ]:
dm_threshold_previous = dm_threshold
        ClassificationError_previous = ClassificationError

        #Fill the history structure fields
        if opt['return_full_history']:
            H['dm_threshold'] = dm_threshold
            H['dm'] = dm
            H['ClassLabel_Proportion'] = ClassLabel_Proportion
            H['DominantClassLabel'] = DominantClassLabel
            H['DominantClassLabel_Proportion'] = DominantClassLabel_Proportion
            H['ClusterClassificationError'] = ClusterClassificationError
            H['ClassificationError'] = ClassificationError
            H['ClassLabelProportion'] = ClassLabel_Proportion
            H['sum1'] = sum1
            H['C'] = C
            H['rmXidx'] = rmXidx
            H['ElapsedTime'] = time.time() - tStart_while
            H_list.append(H)

            _,H = initializeStructures(opt)
        # Check the stop criterion
        if i>=opt['iteration_threshold']:
            #print('WARNING: The maximum number of iterations has been reached.')
            return -1 
    
    return rmH_list, H_list


Local Functions

In [ ]:
def check_separability_criteria(k,nCl, dm_threshold, X, ClassLabel_Num, ClassLabel_Indices, Indices, opt):
    dm = []
    if opt['return_full_dm']:
        dm = []
        for c in range(k):
        #print(f'DM DO CLUSTER C: {c}')
        #print(f'Indices de C: {Indices[c]}')
            for a in range(0,nCl-1):
                Na = ClassLabel_Num[c,a]
                #print(f'Na: {Na}')
                aIdx = Indices[c][ClassLabel_Indices[c,a]]
                for b in range(a+1,nCl):
                    Nb = ClassLabel_Num[c,b]
                    bIdx = Indices[c][ClassLabel_Indices[c,b]]
                    #print(f'Nb: {Nb}')
                    if Na > 2 and Nb > 2:
                        dm.append(d_cs.Divergence_Measure(
                            X[aIdx, :], 
                            X[bIdx, :], 
                            opt['dm_case']
                        ))  
                
        #print("DM " + str(dm[-1]))
        # Yes, there is separable region. Set k = k + 1
        for j in range(len(dm)):
            if dm[j] >= dm_threshold:
                cst = 1
                break
            else:
                cst = 0
        if len(dm) == 0:
            cst = 0
        return cst, dm
    else:
        for c in range(k):
        #print(f'DM DO CLUSTER C: {c}')
        #print(f'Indices de C: {Indices[c]}')
            for a in range(0,nCl-1):
                Na = ClassLabel_Num[c,a]
                #print(f'Na: {Na}')
                aIdx = Indices[c][ClassLabel_Indices[c,a]]
                for b in range(a+1,nCl):
                    Nb = ClassLabel_Num[c,b]
                    bIdx = Indices[c][ClassLabel_Indices[c,b]]
                    #print(f'Nb: {Nb}')
                    if Na > 2 and Nb > 2:
                        dm = d_cs.Divergence_Measure(
                            X[aIdx, :], 
                            X[bIdx, :], 
                            opt['dm_case']
                        )
                        if dm >= dm_threshold:
                            return 1, dm
        return 0, dm


def initializeStructures(opt=None):
    rmH = {
        'i': [],
        'k': [],
        'C': [],
        'rmCidx': [],
        'rmC': [],
        'dm_threshold': [],
        'dm': [],
        'rmDominantClassLabel': [],
        'rmDominantClassLabel_Proportion': [],
        'sum1': [],
        'rmXidx': [],
    }
    if opt['return_full_history']:
        H = {
            'i': [],
            'k': [],
            'C': [],
            'dm_threshold': [],
            'dm': [],
            'ClassLabelProportion': [],
            'DominantClassLabel': [],
            'DominantClassLabel_Proportion': [],
            'ClusterClassificationError': [],
            'ClassificationError': [],
            'sum1': [],
            'rmXidx': [],
            'ElapsedTime': [],
        }
    else:
        H = {}

    return rmH, H
